# Degradation Model Development

In [6]:
import os
import sys
import pickle
import h5py
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import math
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

base_dir = os.path.dirname(os.getcwd())
print(base_dir)
sys.path.insert(1, base_dir)
from package.api import DB as api
import package.utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2


print(tf.__version__)
gpu = tf.config.list_physical_devices('GPU')
has_gpu = gpu[0][0].split(':')[1] == 'GPU'
print(f"[INFO] GPU?: <{has_gpu}> {gpu}")
if(has_gpu):
    tf.config.experimental.set_memory_growth(gpu[0], True)

/home/oem/phm2021_data_challenge
2.7.0
[INFO] GPU?: <True> [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [9]:
# THESE ARE YOUR CREDENTIALS IN PLAIN TEXT!
params = utils.get_aws_secret("/secret/cmapss")
#print(params)
db, cur =  api.connect(params)
db.set_session(autocommit=True)
del(params)

[INFO] connecting to db.
[INFO] connected.


## First, go thru this with the 1 degradation source (DS01-005)

### Get units

In [10]:
units = api._get_units(group_id=[1,2], 
                       Fc=[1,2,3],
                       datasets=['DS01-005'],
                       db=db)
units.head()

,id,serial_number,age,eol,rul,group_id,Fc,unit,dataset
0,1,bB6BMGAz,0.0,100.0,100.0,1,1,1,DS01-005
1,2,AzPb1tFB,0.0,75.0,75.0,1,3,2,DS01-005
2,3,lrHrQ11z,0.0,100.0,100.0,1,2,3,DS01-005
3,4,ANAEtKCP,0.0,95.0,95.0,1,1,4,DS01-005
4,5,oU8DYEXh,0.0,89.0,89.0,1,3,5,DS01-005


### Get data

In [22]:
downsample=5

summary_data = api._get_data(units=list(units.id),
                             downsample=downsample,
                             table='summary_tb',
                             db=db)
summary_data.head()

,id,asset_id,cycle,hs,alt,Mach,TRA,T2
0,5,1,1,1.0,3043.0,0.37762,70.39989,522.30061
1,10,1,1,1.0,3081.0,0.37945,70.39989,522.30212
2,15,1,1,1.0,3119.0,0.38146,70.57567,522.31835
3,20,1,1,1.0,3153.0,0.38449,70.57567,522.42816
4,25,1,1,1.0,3190.0,0.38758,70.57567,522.53470


In [23]:
telemetry_data = api._get_data(units=list(units.id),
                               downsample=downsample,
                               table='telemetry_tb',
                               db=db)
telemetry_data.head()

Execution failed on sql 'select tb.* from (select *, row_number() over() rn from telemetry_tb) tb where asset_id in (1, 2, 3, 4, 5, 6, 7, 8, 9, 10) and tb.rn % 5 = 0;': column "asset_id" does not exist
LINE 1: ...ow_number() over() rn from telemetry_tb) tb where asset_id i...
                                                             ^

None


Traceback (most recent call last):
  File "/home/oem/anaconda3/envs/tfgpu/lib/python3.8/site-packages/pandas/io/sql.py", line 2056, in execute
    cur.execute(*args, **kwargs)
psycopg2.errors.UndefinedColumn: column "asset_id" does not exist
LINE 1: ...ow_number() over() rn from telemetry_tb) tb where asset_id i...
                                                             ^


The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/oem/phm2021_data_challenge/package/api.py", line 66, in execute
    return pd.read_sql_query(sql_query, database)
  File "/home/oem/anaconda3/envs/tfgpu/lib/python3.8/site-packages/pandas/io/sql.py", line 436, in read_sql_query
    return pandas_sql.read_query(
  File "/home/oem/anaconda3/envs/tfgpu/lib/python3.8/site-packages/pandas/io/sql.py", line 2116, in read_query
    cursor = self.execute(*args)
  File "/home/oem/anaconda3/envs/tfgpu/lib/python3.8/site-packages/pandas/io/sql.py", line 20

KeyError: "['rn'] not found in axis"